#1 $$u"=f(x) u(0)=u(1)=0$$

In [5]:
import numpy as np

def f(x):
    return np.where((x >= 0.4) & (x <= 0.6), 1.0, 0.0)

def u_exact(x):
    x = np.asarray(x)
    u = np.zeros_like(x)

    mask1 = (x >= 0.0) & (x <= 0.4)
    u[mask1] = -0.1 * x[mask1]

    mask2 = (x > 0.4) & (x <= 0.6)
    u[mask2] = 0.5 * x[mask2]**2 - 0.5 * x[mask2] + 2/25

    mask3 = (x > 0.6) & (x <= 1.0)
    u[mask3] = 0.1 * x[mask3] - 0.1

    return u

def solve_fd(n):
    h = 1.0 / n
    x = np.linspace(0.0, 1.0, n + 1)

    x_inner = x[1:-1]
    m = n - 1
    main_diag = -2.0 * np.ones(m)
    off_diag  = 1.0 * np.ones(m - 1)
    A = (np.diag(main_diag) +
         np.diag(off_diag, k=1) +
         np.diag(off_diag, k=-1)) / h**2

    b = f(x_inner)

    u_inner = np.linalg.solve(A, b)

    u_h = np.zeros(n + 1)
    u_h[1:-1] = u_inner

    u_ex = u_exact(x)

    err = u_h - u_ex
    err_max = np.max(np.abs(err))
    err_l2  = np.sqrt(h * np.sum(err**2))

    return x, u_h, err_max, err_l2

Ns = [20, 40, 80, 160, 320]

print(f"{'n':>5}  {'h':>8}  {'max error':>12}  {'L2 error':>12}")
for n in Ns:
    x, uh, emax, eL2 = solve_fd(n)
    h = 1.0 / n
    print(f"{n:5d}  {h:8.4f}  {emax:12.4e}  {eL2:12.4e}")

    n         h     max error      L2 error
   20    0.0500    2.0000e-03    1.1726e-03
   40    0.0250    1.0000e-03    5.7960e-04
   80    0.0125    5.0000e-04    2.8896e-04
  160    0.0063    2.5000e-04    1.4437e-04
  320    0.0031    1.2500e-04    7.2173e-05


In [8]:
prev_emax = None
print("\nConvergence (max error ratio):")
for n in Ns:
    _, _,   emax , _ = solve_fd(n)
    if prev_emax is not None:
        print(f"n = {n:4d},  ratio ≈ {prev_emax/emax:8.3f}")
    prev_emax = emax
print("\nConvergence (L2 error ratio):")
prev_L2 = None
for n in Ns:
    _, _, _, eL2 = solve_fd(n)
    if prev_L2 is not None:
        print(f"n = {n:4d},  ratio ≈ {prev_L2/eL2:8.3f}")
    prev_L2 = eL2


Convergence (max error ratio):
n =   40,  ratio ≈    2.000
n =   80,  ratio ≈    2.000
n =  160,  ratio ≈    2.000
n =  320,  ratio ≈    2.000

Convergence (L2 error ratio):
n =   40,  ratio ≈    2.023
n =   80,  ratio ≈    2.006
n =  160,  ratio ≈    2.001
n =  320,  ratio ≈    2.000


In [7]:
def f_smooth(x):
    return np.sin(np.pi * x)

def u_exact_smooth(x):
    return -np.sin(np.pi * x) / (np.pi**2)
def solve_fd_smooth(n):
    h = 1.0 / n
    x = np.linspace(0.0, 1.0, n + 1)
    x_inner = x[1:-1]
    m = n - 1

    main_diag = -2.0 * np.ones(m)
    off_diag  = 1.0 * np.ones(m - 1)
    A = (np.diag(main_diag) +
         np.diag(off_diag, k=1) +
         np.diag(off_diag, k=-1)) / h**2

    b = f_smooth(x_inner)

    u_inner = np.linalg.solve(A, b)

    u_h = np.zeros(n + 1)
    u_h[1:-1] = u_inner

    u_ex = u_exact_smooth(x)
    err = u_h - u_ex
    err_max = np.max(np.abs(err))
    err_l2  = np.sqrt(h * np.sum(err**2))
    return x, u_h, err_max, err_l2


Ns = [20, 40, 80, 160, 320]

print("Smooth test (u'' = sin(pi x))")
print(f"{'n':>5}  {'h':>8}  {'max error':>12}  {'L2 error':>12}")
for n in Ns:
    x, uh, emax, eL2 = solve_fd_smooth(n)
    h = 1.0 / n
    print(f"{n:5d}  {h:8.4f}  {emax:12.4e}  {eL2:12.4e}")

print("\nConvergence ratio (max error):")
prev_emax = None
for n in Ns:
    _, _, emax, _ = solve_fd_smooth(n)
    if prev_emax is not None:
        print(f"n = {n:4d},  ratio ≈ {prev_emax/emax:8.3f}")
    prev_emax = emax

print("\nConvergence ratio (L2 error):")
prev_L2 = None
for n in Ns:
    _, _, _, eL2 = solve_fd_smooth(n)
    if prev_L2 is not None:
        print(f"n = {n:4d},  ratio ≈ {prev_L2/eL2:8.3f}")
    prev_L2 = eL2

Smooth test (u'' = sin(pi x))
    n         h     max error      L2 error
   20    0.0500    2.0859e-04    1.4750e-04
   40    0.0250    5.2099e-05    3.6840e-05
   80    0.0125    1.3022e-05    9.2078e-06
  160    0.0063    3.2553e-06    2.3018e-06
  320    0.0031    8.1381e-07    5.7545e-07

Convergence ratio (max error):
n =   40,  ratio ≈    4.004
n =   80,  ratio ≈    4.001
n =  160,  ratio ≈    4.000
n =  320,  ratio ≈    4.000

Convergence ratio (L2 error):
n =   40,  ratio ≈    4.004
n =   80,  ratio ≈    4.001
n =  160,  ratio ≈    4.000
n =  320,  ratio ≈    4.000


#2
$$u''-2u'+u=1,u(0)=0,u'(1)=1$$

In [11]:
def u_exact(x):
    e = np.e
    C2 = (1.0 + e) / (2.0 * e)
    return (-1.0 + C2 * x) * np.exp(x) + 1.0

def solve_fd(n):

    h = 1.0 / n
    x = np.linspace(0.0, 1.0, n + 1)

    m = n
    A = np.zeros((m, m))
    b = np.zeros(m)

    for j in range(1, n):
        row = j - 1
        if j == 1:
            A[row, 0] = -2.0 + h**2
            if n >= 2:
                A[row, 1] = 1.0 - h
            b[row] = h**2
        elif j == n - 1:
            A[row, row - 1] = 1.0 + h
            A[row, row]     = -2.0 + h**2
            A[row, row + 1] = 1.0 - h
            b[row] = h**2
        else:
            A[row, row - 1] = 1.0 + h
            A[row, row]     = -2.0 + h**2
            A[row, row + 1] = 1.0 - h
            b[row] = h**2
    row = n - 1
    A[row, n - 3] = 1.0
    A[row, n - 2] = -4.0
    A[row, n - 1] = 3.0
    b[row] = 2.0 * h
    u_inner = np.linalg.solve(A, b)

    u_h = np.zeros(n + 1)
    u_h[0] = 0.0
    u_h[1:] = u_inner

    u_ex = u_exact(x)
    err = u_h - u_ex

    err_max = np.max(np.abs(err))
    err_L2  = np.sqrt(h * np.sum(err**2))

    return x, u_h, err_max, err_L2

Ns = [10, 20, 40, 80, 160]

print(f"{'n':>5}  {'h':>8}  {'max error':>12}  {'L2 error':>12}")
for n in Ns:
    x, u_h, emax, eL2 = solve_fd(n)
    h = 1.0 / n
    print(f"{n:5d}  {h:8.4f}  {emax:12.4e}  {eL2:12.4e}")

print("\nConvergence ratio (max error):")
prev = None
for n in Ns:
    _, _, emax, _ = solve_fd(n)
    if prev is not None:
        print(f"n = {n:4d},  ratio ≈ {prev/emax:8.3f}")
    prev = emax

print("\nConvergence ratio (L2 error):")
prev = None
for n in Ns:
    _, _, _, eL2 = solve_fd(n)
    if prev is not None:
        print(f"n = {n:4d},  ratio ≈ {prev/eL2:8.3f}")
    prev = eL2

    n         h     max error      L2 error
   10    0.1000    4.9894e-03    2.3004e-03
   20    0.0500    1.3278e-03    5.7479e-04
   40    0.0250    3.4328e-04    1.4388e-04
   80    0.0125    8.7327e-05    3.6010e-05
  160    0.0063    2.2026e-05    9.0084e-06

Convergence ratio (max error):
n =   20,  ratio ≈    3.758
n =   40,  ratio ≈    3.868
n =   80,  ratio ≈    3.931
n =  160,  ratio ≈    3.965

Convergence ratio (L2 error):
n =   20,  ratio ≈    4.002
n =   40,  ratio ≈    3.995
n =   80,  ratio ≈    3.996
n =  160,  ratio ≈    3.997


#3
$$u^{''}=\sin(2\pi x)$$

In [12]:
def u_exact(x):
    pi = np.pi
    return x/(2*pi) - np.sin(2*pi*x)/(4*pi**2)

def solve_fd_neumann(n):
    h = 1.0 / n
    x = np.linspace(0.0, 1.0, n + 1)
    m = n + 1

    A = np.zeros((m, m))
    b = np.zeros(m)
    A[0, 0] = -3.0
    A[0, 1] =  4.0
    A[0, 2] = -1.0
    b[0]    =  0.0
    for j in range(1, n):
        A[j, j-1] =  1.0
        A[j, j]   = -2.0
        A[j, j+1] =  1.0
        b[j]      = h**2 * np.sin(2*np.pi * x[j])

    A[n, n-2] =  1.0
    A[n, n-1] = -4.0
    A[n, n]   =  3.0
    b[n]      =  0.0

    u_num, *_ = np.linalg.lstsq(A, b, rcond=None)

    return x, u_num

def compute_error(n):
    x, u_num = solve_fd_neumann(n)

    u_ex_base = u_exact(x)

    shift = u_num[0] - u_ex_base[0]
    u_ex = u_ex_base + shift

    err = u_num - u_ex
    h = 1.0 / n

    err_max = np.max(np.abs(err))
    err_L2  = np.sqrt(h * np.sum(err**2))

    return x, u_num, err_max, err_L2

Ns = [20, 40, 80, 160, 320]

print(f"{'n':>5}  {'h':>8}  {'max error':>12}  {'L2 error':>12}")
for n in Ns:
    _, _, emax, eL2 = compute_error(n)
    h = 1.0 / n
    print(f"{n:5d}  {h:8.4f}  {emax:12.4e}  {eL2:12.4e}")

print("\nConvergence ratio (max error):")
prev = None
for n in Ns:
    _, _, emax, _ = compute_error(n)
    if prev is not None:
        print(f"n = {n:4d},  ratio ≈ {prev/emax:8.3f}")
    prev = emax

print("\nConvergence ratio (L2 error):")
prev = None
for n in Ns:
    _, _, _, eL2 = compute_error(n)
    if prev is not None:
        print(f"n = {n:4d},  ratio ≈ {prev/eL2:8.3f}")
    prev = eL2

    n         h     max error      L2 error
   20    0.0500    6.4143e-03    3.8995e-03
   40    0.0250    1.6280e-03    9.7224e-04
   80    0.0125    4.0855e-04    2.4179e-04
  160    0.0063    1.0223e-04    6.0232e-05
  320    0.0031    2.5564e-05    1.5027e-05

Convergence ratio (max error):
n =   40,  ratio ≈    3.940
n =   80,  ratio ≈    3.985
n =  160,  ratio ≈    3.996
n =  320,  ratio ≈    3.999

Convergence ratio (L2 error):
n =   40,  ratio ≈    4.011
n =   80,  ratio ≈    4.021
n =  160,  ratio ≈    4.014
n =  320,  ratio ≈    4.008


#5
$$ϵu''+(1+ϵ)u'+u=0,u(0)=u(1)=0$$

In [13]:
eps = 0.01

def u_exact(x, eps=eps):
    a = eps
    b = 1.0 + eps
    c = 1.0
    disc = b**2 - 4*a*c
    r1 = (-b + np.sqrt(disc)) / (2*a)
    r2 = (-b - np.sqrt(disc)) / (2*a)
    denom = np.exp(r1) - np.exp(r2)
    return (np.exp(r1 * x) - np.exp(r2 * x)) / denom

def solve_fd(n, eps=eps):
    h = 1.0 / n
    x = np.linspace(0.0, 1.0, n + 1)
    m = n - 1
    A = np.zeros((m, m))
    b = np.zeros(m)
    a_minus = eps / h**2 - (1.0 + eps) / (2.0 * h)
    a_zero  = -2.0 * eps / h**2 + 1.0
    a_plus  = eps / h**2 + (1.0 + eps) / (2.0 * h)
    for j in range(1, n):
        row = j - 1
        if j - 1 >= 1:
            A[row, row - 1] = a_minus
        else:
            pass

        A[row, row] = a_zero

        if j + 1 <= n - 1:
            A[row, row + 1] = a_plus
        else:

            b[row] = -a_plus * 1.0

    u_inner = np.linalg.solve(A, b)

    u_h = np.zeros(n + 1)
    u_h[0] = 0.0
    u_h[1:n] = u_inner
    u_h[n] = 1.0

    u_ex = u_exact(x, eps)
    err = u_h - u_ex

    err_max = np.max(np.abs(err))
    err_L2 = np.sqrt(h * np.sum(err**2))

    return x, u_h, err_max, err_L2

Ns = [20, 40, 80, 160, 320]

print(f"{'n':>5}  {'h':>8}  {'max error':>14}  {'L2 error':>14}")
for n in Ns:
    _, _, emax, eL2 = solve_fd(n)
    h = 1.0 / n
    print(f"{n:5d}  {h:8.4f}  {emax:14.6e}  {eL2:14.6e}")

print("\nConvergence ratio (max error):")
prev = None
for n in Ns:
    _, _, emax, _ = solve_fd(n)
    if prev is not None:
        print(f"n = {n:4d}, ratio ≈ {prev/emax:8.3f}")
    prev = emax

print("\nConvergence ratio (L2 error):")
prev = None
for n in Ns:
    _, _, _, eL2 = solve_fd(n)
    if prev is not None:
        print(f"n = {n:4d}, ratio ≈ {prev/eL2:8.3f}")
    prev = eL2

    n         h       max error        L2 error
   20    0.0500    1.253020e+00    3.137589e-01
   40    0.0250    5.461865e-01    8.642497e-02
   80    0.0125    1.565407e-01    2.003608e-02
  160    0.0063    3.397533e-02    4.700161e-03
  320    0.0031    8.458388e-03    1.148593e-03

Convergence ratio (max error):
n =   40, ratio ≈    2.294
n =   80, ratio ≈    3.489
n =  160, ratio ≈    4.607
n =  320, ratio ≈    4.017

Convergence ratio (L2 error):
n =   40, ratio ≈    3.630
n =   80, ratio ≈    4.313
n =  160, ratio ≈    4.263
n =  320, ratio ≈    4.092
